In [1]:
import numpy as np
import pandas as pd
import boto3
from decimal import Decimal
import datetime as dt

In [10]:
dynamodb = boto3.resource('dynamodb', region_name='us-west-2')

# Insert Movies Table

In [27]:
movies = pd.read_csv('./movies.csv', dtype={0:'int64',1:'object', 2:'object'})
links = pd.read_csv('./links.csv', dtype={0:'int64',1: 'object', 2:'object'})
links.set_index('movieId', inplace=True)

In [31]:
movies.columns

Index(['movieId', 'title', 'genres'], dtype='object')

In [29]:
with dynamodb.Table('movies').batch_writer() as batch:
    for index, movie in movies.iterrows():
        data = {
            "movieId": movie["movieId"],
            "title": movie["title"],
            "imdbId": links.loc[movie["movieId"],"imdbId"],
            "url": "https://www.imdb.com/title/tt" + str(links.loc[movie["movieId"],"imdbId"]),
            "genres": set(movie["genres"].split("|"))
        }
        batch.put_item(Item=data)
    

# Insert Ratings Table

In [13]:
ratings = pd.read_csv('./personalize_user_interaction.csv')
ratings.head()

,USER_ID,ITEM_ID,TIMESTAMP
0,1,2,1112486027
1,1,29,1112484676
2,1,32,1112484819
3,1,47,1112484727
4,1,50,1112484580


In [ ]:
with dynamodb.Table('movie_likes').batch_writer() as batch:
    for index, rating in ratings.iterrows():
        batch.put_item(Item={
            "userId": rating["USER_ID"].item(),
            "movieId": rating["ITEM_ID"].item(),
            "timestamp": rating['TIMESTAMP'].item()
        })